# **Laboratorio 7**

- Derek Arreaga - 22537
- Paula Barillas - 22764
- Mónica Salvatierra - 22249

Link del respositorio: https://github.com/FabianKel/ModSim-LAB7

## **Parte 2**

### **Teoría**

**1. Imagine que usted ha sido contratado para modelar el sistema de tráfico de una gran ciudad con el fin de evaluar el impacto de un nuevo sistema de metro. ¿Por qué un modelo puramente MBA, DES o DS sería, por sí solo, insuficiente para esta tarea? Describa cómo diseñaría usted un modelo híbrido para este problema, identificando qué componente del sistema sería modelado con cada paradigma y cómo interactuarían entre sí.**


En este caso el MBA modela la heterogeneidad y la toma de decisiones individual como por ejemplo preferencias, horarios, adaptación al tiempo de espera pero escalarlo a una ciudad entera y obtener medidas agregadas fiables consideramos que puede llegar a  ser muy costoso computacionalmente y complejo de calibrar; ahora si evaluamos a DES, podemos considerarlo que es ideal para procesos operativos y colas (embarque, control de acceso, capacidad de andenes) pero no modela fácilmente las decisiones adaptativas y la emergencia social que surge de millones de agentes individuales; y el SD es útil para estudiar retroalimentaciones y dinámicas agregadas a mediano-largo plazo (como puede ser cambios de demanda, uso del suelo, efectos de políticas) pero pierde detalle micro y variabilidad individual. Por eso creemos que es mejor un modelo híbrido en el que el nivel micro sea un MBA que represente a viajeros y vehículos con atributos individuales y reglas de decisión, por otro lado el nivel operativo sea un módulo DES que gestione recursos discretos (como los trenes, andenes, colas y procesos de embarque) y por último el nivel estratégico sea un modelo SD que capture variables agregadas y bucles de retroalimentación lentos (percepción del servicio, migración residencial, cambios de oferta). En donde la interacción sería que los agentes MBA generan demanda (llegadas a estaciones) que el DES procesa como entidades en colas y servicios; el DES devuelve indicadores operativos (tiempos de espera, tasas de rechazo) que modifican el comportamiento de los agentes; y el SD consume agregados (ridership diario, tiempo medio de viaje) para actualizar variables lentas y emitir políticas o ajustes (tarifas, inversión) que retroalimentan al MBA y al DES. Un ejemplo para entender mejor podría ser una pasajera decide usar o no el metro según su experiencia de espera (MBA ↔ DES) y, a largo plazo, mejores niveles de servicio pueden atraer vivienda cerca de estaciones, incrementando demanda (SD).


**2. En el 'Patrón 1: MBA → DES', la transición de un agente a una entidad debe ser fluida. Describa a nivel conceptual y técnico (¿qué información específica se debe transferir?) cómo se realizaría esta transición en una herramienta como AnyLogic, o de forma más abstracta, en un marco de trabajo en Python que combinara las bibliotecas Mesa (para MBA) y SimPy (para DES).**

Se entiende que la transición de agente a entidad debe incluir al menos un identificador único, el tipo de agente, el plan (como puede ser el origen, destino, hora estimada de llegada), los atributos operativos relevantes (por ejemplo tamaño del grupo, prioridad o movilidad reducida), el estado de la etapa del viaje y, para reproducibilidad, la semilla o estado aleatorio asociado. En AnyLogic esto se implementa de forma nativa convirtiendo un Agent en una 'entity' del Process Modeling Library (bloques enter/service), trasladando atributos con setters y manteniendo un mapeo entre el agent y la entity para devolver resultados operativos. En Python, una arquitectura práctica usa Mesa para la lógica de agentes y SimPy para las colas y recursos asi que cuando un agente Mesa llega a una estación, se crea y encola una entidad SimPy (representando un objeto ligero con sus atributos) en la cola del recurso. Ya que SimPy procesa la entidad y, al finalizar actualiza el objeto y notifica al agente por ejemplo, liberando un evento o actualizando una bandera compartida para que el agente actualice su plan. Técnicamente hay que acordar la unidad de tiempo y sincronizar relojes, para lograr mantener correspondencia ID↔objeto, asimismo compartir la semilla aleatoria para reproducibilidad, manejar entidades fallidas y diseñar la interfaz (contrato) entre módulos. Basicamente es como un pasajero (agente) deja su 'ticket' en la cola (entidad), el módulo DES decide si sube o no y devuelve el tiempo de espera que el agente usa para replanificar o elegir otro modo.

**3. El 'Patrón 3: DS dentro de un Agente' es un claro ejemplo de modelado multi-escala. Proponga usted otro escenario (fuera del ámbito biológico), por ejemplo, en economía o sociología, donde este patrón sería de gran utilidad. ¿Qué representaría el agente (MBA) y qué dinámica capturaría su modelo de Dinámica de Sistemas (DS) interno?**

Consideramos que un escenario claro es una red de tiendas minoristas en la que cada tienda es un agente del MBA y dentro de cada agente existe un pequeño modelo SD que describe variables continuas como inventario, caja y flujos de reabastecimiento con retrasos. En este caso el MBA captura clientes heterogéneos que se desplazan entre tiendas, ya que responden a promociones y generan demanda local; el DS interno modela los stocks de inventario, los flujos de ventas y los retrasos de reposición que, a su vez, condicionan decisiones de precios o promociones dentro de la tienda. Este patrón permite capturar emergencias como por ejemplo, una promoción en la tienda "la bendición" reduce el inventario (dinámica interna) y provoca que clientes se desplacen a la tienda "el chino" provocando efecto emergente del MBA en la red, en donde el comportamiento no sería visible si las tiendas fueran representadas sólo por ecuaciones agregadas o por simples agentes sin dinámica interna.

**4. La combinación de paradigmas de modelado presenta desafíos significativos. Desde su perspectiva, ¿cuáles cree usted que son los dos mayores desafíos al construir y validar un modelo híbrido? Considere aspectos como la calibración de parámetros entre los diferentes componentes, la complejidad del software y la validación de los resultados emergentes.**

Consideramos que la combinación de paradigmas conlleva varios retos siendo dos de los más críticos eñ primero sería la calibración y coherencia entre componentes heterogéneos y segundo la complejidad computacional y la atribución de causación en comportamientos emergentes. Ya que la calibración es difícil porque parámetros que viven en escalas distintas por ejemplo, tiempos medios de espera del DES, elasticidades de demanda en SD y reglas individuales en el MBA deben armonizarse; ya que una mitigación práctica es calibrar submódulos de forma independiente con datos específicos y luego realizar una calibración conjunta usando métodos aproximados por ejemplo ABC o protocolos tipo CaliPro, junto con análisis de sensibilidad e identificación de parámetros no identificables y una documentación explícita de los contratos de datos entre módulos. En el caso del coste computacional y la sincronización se atacan con co-simulación por pasos temporales bien definidos, haciendo uso de modelos surrogate o aproximados para la exploración y paralelización, y validación por capas, ya que  primero validaría submodelos con datos observables y luego validaria emergentes concretos un ejemplo puede ser la distribución horaria de viajes. En el caso de la práctica se debe además cuantificar incertidumbre mediante escenarios y comparar salidas del modelo con métricas observables como conteos de pasajeros, tiempos GPS, encuestas para detectar discrepancias que apunten a problemas en las interfaces.



## **Referencias**

1. Achachlouei, M. A., & Hilty, L. (2015). System Dynamics vs. agent-based modeling—comparing models and approaches : A literature review and a transformation procedure. https://www.diva-portal.org/smash/get/diva2:844069/FULLTEXT01.pdf

2. Bastarianto, F. F., Hancock, T. O., Choudhury, C. F., & Manley, E. (2023). Agent-based models in urban transportation: review, challenges, and opportunities. European Transport Research Review, 15(1). https://doi.org/10.1186/s12544-023-00590-5

3. Nanda, P., & Kirschner, D. E. (2023). Calibration methods to fit parameters within complex biological models. Frontiers in Applied Mathematics and Statistics, 9. https://doi.org/10.3389/fams.2023.1256443